# Final Project
###### Karina Ruban

## Task 1. Data cleaning and preparation:
##### 1. Remove duplicates and irrelevant columns.
##### 2. Process missing values accordingly.
##### 3. Convert data types for columns such as dates and numeric values

### In this project, the cleaning and data preparation was done according to a certain algorithm to unify the process and make it easier to correct later.
#### **Algorithm**:
##### 1. Checking and deleting duplicates
##### 2. Analysis and processing of missing values
##### 3. Converting columns to correct types
##### 4. Check for incorrect or illogical values
##### 5. ID Uniqueness Check
##### 6. Processing of categorical features
##### 7. Check for logical consistency
##### 8. Creating flags (if needed)

In [1]:
#import libraries & loadibg datasets

import pandas as pd
import numpy as np
import pickle
from datetime import datetime, time

calls_path = 'C:/Users/79607/Documents/Python/tasks/Calls.xlsx'
calls = pd.read_excel(calls_path)

contacts_path = 'C:/Users/79607/Documents/Python/tasks/Contacts.xlsx'
contacts = pd.read_excel(contacts_path)

deals_path = 'C:/Users/79607/Documents/Python/tasks/Deals.xlsx'
deals = pd.read_excel(deals_path)

spend_path = 'C:/Users/79607/Documents/Python/tasks/Spend.xlsx'
spend = pd.read_excel(spend_path)

## Cleaning of dataset Calls

In [2]:
# 1. info about dataset Calls & checking structure

calls.info()
print(calls.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  int64  
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  float64
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95791 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Dialled Number              0 non-null      float64
 8   Outgoing Call Status        86875 non-null  object 
 9   Scheduled in CRM            86875 non-null  float64
 10  Tag                         0 non-null      float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.0+ MB
                    Id   Call Start Time Call Owner Name     CONTACTID  \
0  58050280000

In [3]:
# Renaming

calls.rename(columns={'Id' : 'Calls_ID', 'Call Start Time': 'Call_Start_Time', 'Call Owner Name' : 'Calls_Manager', 'CONTACTID' : 'Contact_ID', 'Call Type' : 'Call_Type', 'Call Duration (in seconds)' : 'Call_Duration_Sec', 'Call Status' : 'Call_Status', 'Outgoing Call Status' : 'Outgoing_Call_Status', 'Scheduled in CRM' : 'Scheduled_In_CRM', 'Dialled Number' : 'Dialled_Number'}, inplace=True)

In [4]:
# Check for duplicates

calls.duplicated().sum() # 0 found

0

In [5]:
# 2. Check for NaN

calls.isnull().sum()

Calls_ID                    0
Call_Start_Time             0
Calls_Manager               0
Contact_ID               3933
Call_Type                   0
Call_Duration_Sec          83
Call_Status                 0
Dialled_Number          95874
Outgoing_Call_Status     8999
Scheduled_In_CRM         8999
Tag                     95874
dtype: int64

In [6]:
# 2.1. Evaluate what a pass means & Make a decision:
# Contact_ID

# some info was missed. ContactID NaN was changed to unique number that belongs nobody to support format and as flag value of not filled cell
# this number (0000000000000000000) will add to readme instructions for futher analysis 

calls['Contact_ID'] = calls['Contact_ID'].fillna(0000000000000000000)

In [7]:
# 2.2. Evaluate what a pass means & Make a decision:
# Call_Duration_Sec

# check if the lack of info is related to a certain call status
calls_with_nan_duration = calls[calls['Call_Duration_Sec'].isnull()]
# print(calls_with_nan_duration['Call_Status'].unique())
# print(calls_with_nan_duration['Outgoing_Call_Status'].unique()) 

# statuses (['Cancelled' 'Overdue' 'Scheduled']) look like there was no calls so duration in fact was 0
# this value (0) will add to readme instructions for futher analysis to using, for instance, as flag

calls['Call_Duration_Sec'] = calls['Call_Duration_Sec'].fillna(0)

In [8]:
# 2.3. Evaluate what a pass means & Make a decision:
# Dialled_Number & Tag

# the whole columns are empty --> delete

calls.drop(columns=['Dialled_Number','Tag'], inplace=True)

In [9]:
# 2.4. Evaluate what a pass means & Make a decision:
# Outgoing_Call_Status

# check if the lack of info is related to a certain call status
outgoing_call_status_calls = calls[calls['Outgoing_Call_Status'].isnull()]
# print(outgoing_call_status_calls['Call_Type'].unique())

# this NaNs are correct because this calls were not outbounds.
# status inbound or smth else in this column is illogical

In [10]:
# 2.5. Evaluate what a pass means & Make a decision:
# Scheduled_In_CRM 

# check if the lack of info is related to a certain call status
scheduled_calls = calls[calls['Scheduled_In_CRM'] == True]
# print(scheduled_calls['Call_Status'].unique())
# print(scheduled_calls['Outgoing_Call_Status'].unique())

nan_calls = calls[calls['Scheduled_In_CRM'].isnull()]
# print(nan_calls['Call_Status'].unique())
# print(nan_calls['Outgoing_Call_Status'].unique())

# statuses ['Received' 'Missed'] are different but no 'scheduled' status means in fact not scheduled --> 0 (not scheduled)
calls['Scheduled_In_CRM'] = calls['Scheduled_In_CRM'].fillna(0)

In [11]:
# 3. Converting columns to correct types

#Dates
calls['Call_Start_Time'] = pd.to_datetime(calls['Call_Start_Time'], errors='coerce', dayfirst=True)

#Strings
calls['Calls_ID'] = calls['Calls_ID'].astype(str)
calls['Contact_ID'] = calls['Contact_ID'].astype(str)

#Numbers
calls['Call_Duration_Sec'] = calls['Call_Duration_Sec'].astype(int)

#Boolean
calls['Scheduled_In_CRM'] = calls['Scheduled_In_CRM'].astype(bool)

#Categories
calls_categorical_columns = ['Calls_Manager', 'Call_Type', 'Call_Status', 'Outgoing_Call_Status']
for col in calls_categorical_columns:
    calls[col] = calls[col].astype('category')

In [12]:
# 4.1. Check for incorrect or illogical values
# deleting spaces & checking mistypes in categories

# just for info to check mistypes. for thousands of unique values other methods are needed, at least creating a dict and checking with it
for col in calls_categorical_columns:
    print(calls[col].value_counts()) 
    
for col in calls_categorical_columns:
    if calls[col].dtype == 'category':
        calls[col] = calls[col].str.strip()
        calls[col] = calls[col].astype('category')

Calls_Manager
Yara Edwards       9059
Julia Nelson       7446
Ian Miller         7215
Charlie Davis      7213
Diana Evans        6857
Ulysses Adams      6085
Amy Green          5982
Nina Scott         5581
Victor Barnes      5439
Kevin Parker       5406
Paula Underwood    4580
Quincy Vincent     4384
Jane Smith         3753
Cara Iverson       3300
John Doe           2986
Ben Hall           2947
Alice Johnson      1251
Mason Roberts      1166
Derek James         948
George King         850
Zachary Foster      523
Eva Kent            498
Fiona Jackson       470
Sam Young           457
Rachel White        441
Xander Dean         304
Ethan Harris        280
Hannah Lee          175
Wendy Clark         162
Bob Brown            99
Oliver Taylor        10
Tina Zhang            5
Laura Quinn           2
Name: count, dtype: int64
Call_Type
Outbound    86875
Missed       5921
Inbound      3078
Name: count, dtype: int64
Call_Status
Attended Dialled              70703
Unattended Dialled            

In [13]:
# 4.2. Check for incorrect or illogical values
# check for negative values in call duration

negative_call_duration = calls[calls['Call_Duration_Sec'] < 0]
# if not negative_call_duration.empty:
#     print(f'Found {len(negative_duration)}calls with negative call duration')
# else:
#     print('Negative call duration not found') # Negative call duration not found

In [14]:
# 4.3. Check for incorrect or illogical values
# check for calls in future

future_calls = calls[calls['Call_Start_Time'] > datetime.now()]
# if not future_calls.empty:
#     print(f'Found {len(future_calls)} call with dates in future')
# else:
#     print('No future call dates found') # No future call dates found

In [15]:
# 5. Authentication of unique id

total_calls_rows = len(calls)
unique_calls_ids = calls['Calls_ID'].nunique()

# if total_calls_rows == unique_calls_ids:
#     print('All ID calls are unique')
# else:
#     print(f'Found duplicates in ID calls. Number of duplicates: {total_calls_rows - unique_calls_ids}') # All ID calls are unique

In [16]:
# 6. Categorical feature processing
# Reducing categories to the last status as actual

calls['Call_Status'] = calls['Call_Status'].str.split().str[-1]
calls['Call_Status'] = calls['Call_Status'].astype('category')
print(calls['Call_Status'].unique())

['Received', 'Dialled', 'Missed', 'Cancelled', 'Unattended', 'Overdue', 'Delay', 'Attended', 'Scheduled']
Categories (9, object): ['Attended', 'Cancelled', 'Delay', 'Dialled', ..., 'Overdue', 'Received', 'Scheduled', 'Unattended']


In [17]:
# 7. Check for logical consistency
# Check that inbounds has not outgoing status

inbound_with_outgoing_status = calls[(calls['Call_Type'] == 'Inbound')&(calls['Outgoing_Call_Status'].notna())]

# if not inbound_with_outgoing_status.empty:
#     print(f'Found {len(inbound_with_outgoing_status)} incoming calls with outgoing call status')
# else:
#     print('Incoming calls do not have outbound status. Logical consistency is respected') # Incoming calls do not have outbound status. Logical consistency is respected


In [18]:
# check info about dataset Calls & structure after cleaning & prep

calls.info()
print(calls.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Calls_ID              95874 non-null  object        
 1   Call_Start_Time       95874 non-null  datetime64[ns]
 2   Calls_Manager         95874 non-null  category      
 3   Contact_ID            95874 non-null  object        
 4   Call_Type             95874 non-null  category      
 5   Call_Duration_Sec     95874 non-null  int32         
 6   Call_Status           95874 non-null  category      
 7   Outgoing_Call_Status  86875 non-null  category      
 8   Scheduled_In_CRM      95874 non-null  bool          
dtypes: bool(1), category(4), datetime64[ns](1), int32(1), object(2)
memory usage: 3.0+ MB
              Calls_ID     Call_Start_Time Calls_Manager  \
0  5805028000000805001 2023-06-30 08:43:00      John Doe   
1  5805028000000768006 2023-06-30 08:46:00 

## Cleanig of dataset Contacts

In [19]:
# 1. info about dataset Contacts & checking structure

contacts.info()
print(contacts.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  18548 non-null  int64 
 1   Contact Owner Name  18548 non-null  object
 2   Created Time        18548 non-null  object
 3   Modified Time       18548 non-null  object
dtypes: int64(1), object(3)
memory usage: 579.8+ KB
                    Id Contact Owner Name      Created Time     Modified Time
0  5805028000000645014       Rachel White  27.06.2023 11:28  22.12.2023 13:34
1  5805028000000872003      Charlie Davis  03.07.2023 11:31  21.05.2024 10:23
2  5805028000000889001          Bob Brown  02.07.2023 22:37  21.12.2023 13:17
3  5805028000000907006          Bob Brown  03.07.2023 05:44  29.12.2023 15:20
4  5805028000000939010         Nina Scott  04.07.2023 10:11  16.04.2024 16:14


In [20]:
# Renaming

contacts.rename(columns={'Id' : 'Contacts_ID', 'Contact Owner Name' : 'Contacts_Manager', 'Created Time' : 'Created_Time', 'Modified Time' : 'Modified_Time'}, inplace=True)

In [21]:
#Check for duplicates

contacts.duplicated().sum() # 0 found

0

In [22]:
# 2. Check for NaN

contacts.isnull().sum() # 0 found

Contacts_ID         0
Contacts_Manager    0
Created_Time        0
Modified_Time       0
dtype: int64

In [23]:
# 3. Converting columns to correct types

# Category
contacts['Contacts_Manager'] = contacts['Contacts_Manager'].astype('category')

# String
contacts['Contacts_ID'] = contacts['Contacts_ID'].astype(str)

# Dates
contacts['Created_Time'] = pd.to_datetime(contacts['Created_Time'], errors='coerce', dayfirst=True)
contacts['Modified_Time'] = pd.to_datetime(contacts['Modified_Time'], errors='coerce', dayfirst=True)

In [24]:
# 4.1. Check for incorrect or illogical values
# deleting spaces & checking mistypes in categories

contacts['Contacts_Manager'] = contacts['Contacts_Manager'].str.strip()
print(contacts['Contacts_Manager'].value_counts()) #just for info to check mistypes. not optimal for billions rows. in future dict may be used not to check all values, only new ones

contacts.isnull().sum()

# probably one value was " " (just space)
contacts['Contacts_Manager'] = contacts['Contacts_Manager'].fillna('Unknown')
contacts['Contacts_Manager'] = contacts['Contacts_Manager'].astype('category')

Contacts_Manager
Charlie Davis      2018
Ulysses Adams      1816
Julia Nelson       1769
Paula Underwood    1487
Quincy Vincent     1416
Nina Scott         1150
Ben Hall           1038
Victor Barnes       967
Cara Iverson        880
Rachel White        782
Jane Smith          754
Bob Brown           685
Ian Miller          684
Diana Evans         678
Yara Edwards        655
Amy Green           621
Eva Kent            365
Kevin Parker        325
Mason Roberts       217
George King         144
Sam Young            37
Alice Johnson        27
Oliver Taylor        19
Zachary Foster        8
Wendy Clark           2
Tina Zhang            2
Derek James           1
Name: count, dtype: int64


In [25]:
# 4.2. Check for incorrect or illogical values
# check for dates

invalid_dates = contacts[contacts['Modified_Time'] < contacts['Created_Time']]

# if not invalid_dates.empty:
#     print(f'Found {len(invalid_dates)} invalid dates')
# else:
#     print('All dates are valid') # All dates are valid

In [26]:
# 5. Authentication of unique id

total_contacts_rows = len(contacts)
unique_contacts_ids = contacts['Contacts_ID'].nunique()

# if total_contacts_rows == unique_contacts_ids:
#     print('All ID contacts are unique')
# else:
#     print(f'Found duplicates in ID contacts. Number of duplicates: {total_contacts_rows - unique_contacts_ids}') # All ID contacts are unique

In [27]:
# check info about dataset Contacts & structure after cleaning & prep

contacts.info()
print(contacts.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Contacts_ID       18548 non-null  object        
 1   Contacts_Manager  18548 non-null  category      
 2   Created_Time      18548 non-null  datetime64[ns]
 3   Modified_Time     18548 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](2), object(1)
memory usage: 454.2+ KB
           Contacts_ID Contacts_Manager        Created_Time  \
0  5805028000000645014     Rachel White 2023-06-27 11:28:00   
1  5805028000000872003    Charlie Davis 2023-07-03 11:31:00   
2  5805028000000889001        Bob Brown 2023-07-02 22:37:00   
3  5805028000000907006        Bob Brown 2023-07-03 05:44:00   
4  5805028000000939010       Nina Scott 2023-07-04 10:11:00   

        Modified_Time  
0 2023-12-22 13:34:00  
1 2024-05-21 10:23:00  
2 2023-12-21 13:17:00  
3 2023-12-29 15:20:

## Cleaning of dataset Deals


In [28]:
# 1. info about dataset Deals & checking structure

deals.info()
print(deals.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21595 entries, 0 to 21594
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   21593 non-null  float64
 1   Deal Owner Name      21564 non-null  object 
 2   Closing Date         14645 non-null  object 
 3   Quality              19340 non-null  object 
 4   Stage                21593 non-null  object 
 5   Lost Reason          16124 non-null  object 
 6   Page                 21593 non-null  object 
 7   Campaign             16067 non-null  object 
 8   SLA                  15533 non-null  object 
 9   Content              14147 non-null  object 
 10  Term                 12454 non-null  object 
 11  Source               21593 non-null  object 
 12  Payment Type         496 non-null    object 
 13  Product              3592 non-null   object 
 14  Education Type       3300 non-null   object 
 15  Created Time         21593 non-null 

In [29]:
# Renaming

deals.rename(columns={'Id' : 'Deals_ID', 'Deal Owner Name' : 'Deals_Manager', 'Closing Date' : 'Closing_Date', 'Lost Reason' : 'Lost_Reason', 'Campaign' : 'Deals_Campaign', 'Payment Type' : 'Payment_Type', 'Education Type' : 'Education_Type', 'Created Time' : 'Created_Time', 'Course duration' : 'Course_Duration', 'Months of study' : 'Months_Of_Study', 'Initial Amount Paid' : 'Initial_Amount_Paid', 'Offer Total Amount' : 'Offer_Total_Amount', 'Contact Name' : 'Contact_Name', 'Level of Deutsch' : 'Level_Of_German'}, inplace=True)

In [30]:
# Check for duplicates

# deals.duplicated().sum() #3 duplicates
deals = deals.drop_duplicates()

In [31]:
# 2. Check for NaN

deals.isnull().sum()
# Closing_Date wiil kepp like that because it may mean that deal was not closed for any reason

Deals_ID                   2
Deals_Manager             31
Closing_Date            6949
Quality                 2254
Stage                      2
Lost_Reason             5470
Page                       2
Deals_Campaign          5526
SLA                     6059
Content                 7446
Term                    9139
Source                     2
Payment_Type           21096
Product                18000
Education_Type         18292
Created_Time               2
Course_Duration        18005
Months_Of_Study        20752
Initial_Amount_Paid    17427
Offer_Total_Amount     17407
Contact_Name              63
City                   19081
Level_Of_German        20341
dtype: int64

In [32]:
# 2.1. Evaluate what a pass means & Make a decision:
# Deals_ID is a key metric --> delete rows without

deals = deals.dropna(subset=['Deals_ID'])

In [33]:
# 2.2. Evaluate what a pass means & Make a decision:
# Contact_Name is a key metric (ID) --> delete rows without

deals = deals.dropna(subset=['Contact_Name'])

In [34]:
# 2.3. Evaluate what a pass means & Make a decision:
# Deals_Manager for unknown reasons is unknown --> unknown. 
# now impossible to say if it is important to keep or not

deals['Deals_Manager'] = deals['Deals_Manager'].fillna('Unknown')

In [35]:
# 2.4. Evaluate what a pass means & Make a decision:
# Quality 

# print(deals['Quality'].unique())

# check if the lack of info is related to a certain stage or even lost reason

quality_nan_deals = deals[deals['Quality'].str.strip().isna()]
# print(quality_nan_deals['Stage'].unique())
# print(quality_nan_deals['Lost_Reason'].unique())

# there is no pattern --> change to Unknown

deals['Quality'] = deals['Quality'].fillna('G - Unknown')

# Quality F is also not clear qithout any comments. 
# check if the lack of info is related to a certain stage or even lost reason to explain the meaning

quality_f_deals = deals[deals['Quality'].str.strip() == 'F']
# if not quality_f_deals.empty:
#     print(f'Found {len(quality_f_deals)} deals with Quality F')
# else:
#     print('Deals with Quality F not found')
# print(quality_f_deals['Stage'].unique())
# print(quality_f_deals['Lost_Reason'].unique())

# there is no pattern --> combine 3 F deals with other not clear statuses (G - Unknown) & rename to F
deals['Quality'] = deals['Quality'].replace('F', 'G - Unknown')
deals['Quality'] = deals['Quality'].replace('G - Unknown', 'F - Unknown')

In [36]:
# 2.5. Evaluate what a pass means & Make a decision:
# Lost_Reason 

# print(deals['Lost_Reason'].unique())

# check if the lack of info is related to a certain stage

deals_nan_lost_reason = deals[deals['Lost_Reason'].isnull()]
# print(deals_nan_lost_reason['Stage'].unique())

# some NaNs are correct because deals were not lost. For lost deals input "unknown" reason

deals.loc[(deals['Stage'].str.strip() == 'Lost') & (deals['Lost_Reason'].isnull()), 'Lost_Reason'] = 'Unknown'

# From FAQ
# Question: There is a "Lost Reason" column and it takes the following values. As I understand it, the duplicate indicates that the transaction was simply tampered with in the system?
# Answer: Yes, it means the leds were two and in 1 is worth a lot just because it is a duplicate

deals = deals[deals['Lost_Reason'] != 'Duplicate']

# categories will be combined later

In [37]:
# 2.6. Evaluate what a pass means & Make a decision:
# Campaign 

# print(deals['Deals_Campaign'].unique()) #not really informative
deals['Deals_Campaign'] = deals['Deals_Campaign'].fillna('Unknown')

In [38]:
# 2.7. Evaluate what a pass means & Make a decision:
# SLA 
# many troubles because of formats
# conversion to string hh:mm:s

new_SLA = []

for value in deals['SLA']:
    if isinstance(value, (time, datetime)):
        new_SLA.append(value.strftime('%H:%M:%S'))
    else:
        new_SLA.append(value)
    
deals['SLA'] = new_SLA

# SLA may depend on the manager. In the absence of information, empty blanks are filled with mode for all data

mode_SLA = deals.loc[deals['SLA'].notnull(), 'SLA'].mode()

if mode_SLA.empty:
    mode_SLA = '00:00:00'
else:
    mode_SLA = mode_SLA[0]

for manager in deals['Deals_Manager'].unique():
    manager_data = deals[deals['Deals_Manager'] == manager]
    manager_mode_SLA = manager_data.loc[manager_data['SLA'].notnull(), 'SLA'].mode()
    
    if not manager_mode_SLA.empty:
        mode_to_fill = manager_mode_SLA[0]
    else:
        mode_to_fill = mode_SLA
    
    deals.loc[(deals['Deals_Manager'] == manager) & (deals['SLA'].isnull()), 'SLA'] = mode_to_fill

# conversion to date

deals['SLA'] = pd.to_timedelta(deals['SLA'])

In [39]:
# 2.8. Evaluate what a pass means & Make a decision:
# Content

# print(deals['Content'].unique())
deals['Content'] = deals['Content'].fillna('Unknown')

In [40]:
# 2.9. Evaluate what a pass means & Make a decision:
# Term

# print(deals['Term'].unique())
deals['Term'] = deals['Term'].fillna('Unknown')

In [41]:
# 2.10. Evaluate what a pass means & Make a decision:
# Payment_Type

# check if the lack of info is related to a certain stage

# print(deals['Payment_Type'].unique())
payment_type_nan = deals[deals['Payment_Type'].isnull()]
nan_payment_stages = payment_type_nan['Stage'].value_counts(dropna=False)
# print(nan_payment_stages)

# lost deals have no payments so NaN is logical value. for other add 'Unknown'

deals.loc[(deals['Stage'].str.strip() != 'Lost') & (deals['Payment_Type'].isnull()), 'Payment_Type'] = 'Unknown'

In [42]:
# 2.11. Evaluate what a pass means & Make a decision:
# Product

# print(deals['Product'].unique())
deals['Product'] = deals['Product'].fillna('Unknown')

In [43]:
# 2.12. Evaluate what a pass means & Make a decision:
# Education_Type

# print(deals['Education_Type'].unique())

# check if the lack of info is related to a certain product

filled_product_education_type = deals[deals['Education_Type'].notna() & deals['Product'].notna()]
correlation_product_education_type = filled_product_education_type.groupby(['Product', 'Education_Type']).size().unstack(fill_value=0)
# print(correlation_product_education_type)

nan_product_education_type = deals[deals['Education_Type'].isnull()]['Product'].value_counts(dropna=False)
# print(nan_product_education_type)

# according to products add mode education type

deals['Education_Type'] = deals.groupby('Product')['Education_Type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

# the rest will be Unknown

deals['Education_Type'] = deals['Education_Type'].fillna('Unknown')

In [44]:
# 2.13. Evaluate what a pass means & Make a decision:
# Course_Duration

print(deals['Course_Duration'].unique())

# check if the lack of info is related to a certain product

filled_product_course_duration = deals[deals['Course_Duration'].notna() & deals['Product'].notna()]
correlation_product_course_duration = filled_product_course_duration.groupby(['Product', 'Course_Duration']).size().unstack(fill_value=0)
# print(correlation_product_course_duration)

nan_product_course_duration = deals[deals['Course_Duration'].isnull()]['Product'].value_counts(dropna=False)
# print(nan_product_course_duration)

# if prodict is unknown, there is no duration

deals.loc[(deals['Product'] == 'Unknown') & (deals['Course_Duration'].isnull()), 'Course_Duration'] = 0

# according to products add mode course duration

deals['Course_Duration'] = deals.groupby('Product')['Course_Duration'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

# the rest is unknown but for support format will be 0

deals['Course_Duration'] = deals['Course_Duration'].fillna(0)

[nan  6. 11.]


In [45]:
# 2.14. Evaluate what a pass means & Make a decision:
# Months_Of_Study

print(deals['Months_Of_Study'].unique())

# check if the lack of info is related to a certain Stage

filled_stage_months = deals[deals['Months_Of_Study'].notna() & deals['Stage'].notna()]
correlation_stage_months = filled_stage_months.groupby(['Stage', 'Months_Of_Study']).size().unstack(fill_value=0)
print(correlation_stage_months)

nan_stage_months = deals[deals['Months_Of_Study'].isnull()]['Stage'].value_counts(dropna=False)
print(nan_stage_months)

# for stage "payment done" add mode month. for others Nan is ok because they were lost or did not start.
# for non-historical data, the month of study can be calculated with now()

deals.loc[(deals['Stage'] == 'Payment Done') & (deals['Months_Of_Study'].isnull()), 'Months_Of_Study'] = deals.loc[deals['Stage'] == 'Payment Done', 'Months_Of_Study'].mode()[0]

# still refilled less than 5% of rows --> drop the column because this amount is not enough to use

deals = deals.drop('Months_Of_Study', axis=1)

[nan  1.  0.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
Months_Of_Study  0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0   9.0   \
Stage                                                                         
Payment Done        1    65   104    94    93    64   102    78    81    61   

Months_Of_Study  10.0  11.0  
Stage                        
Payment Done       42    45  
Stage
Lost                         13967
Call Delayed                  2236
Registered on Webinar         2069
Waiting For Payment            324
Qualificated                   128
Registered on Offline Day       85
Need to Call - Sales            33
Need To Call                    31
Test Sent                       25
Need a consultation             23
Payment Done                    16
New Lead                         5
Free Education                   1
Name: count, dtype: int64


In [46]:
# 2.15. Evaluate what a pass means & Make a decision:
# Initial_Amount_Paid

# print(deals['Initial_Amount_Paid'].unique())

# strings were found

deals['Initial_Amount_Paid'] = deals['Initial_Amount_Paid'].astype(str).str.replace('€ ', '', regex=False)
deals['Initial_Amount_Paid'] = deals['Initial_Amount_Paid'].str.replace('.', '', regex=False)
deals['Initial_Amount_Paid'] = deals['Initial_Amount_Paid'].str.replace(',', '.', regex=False)

# print(deals['Initial_Amount_Paid'].unique())

# string 'nan' was found

deals['Initial_Amount_Paid'] = deals['Initial_Amount_Paid'].astype(str).str.lower().str.strip().replace('nan', np.nan)
deals['Initial_Amount_Paid'] = pd.to_numeric(deals['Initial_Amount_Paid'], errors='coerce')

# all have different conditions --> not enough info to refill but to support format will add flag as 1000000

deals['Initial_Amount_Paid'] = deals['Initial_Amount_Paid'].fillna(1000000)

# print(deals['Initial_Amount_Paid'].unique())

In [47]:
# 2.16. Evaluate what a pass means & Make a decision:
# Offer_Total_Amount

# print(deals['Offer_Total_Amount'].unique())

# found strings

deals['Offer_Total_Amount'] = deals['Offer_Total_Amount'].astype(str).str.replace('€ ', '', regex=False)
deals['Offer_Total_Amount'] = deals['Offer_Total_Amount'].str.replace('.', '', regex=False)
deals['Offer_Total_Amount'] = deals['Offer_Total_Amount'].str.replace(',', '.', regex=False)

# print(deals['Offer_Total_Amount'].unique())

# string 'nan' was found

deals['Offer_Total_Amount'] = deals['Offer_Total_Amount'].astype(str).str.lower().str.strip().replace('nan', np.nan)
deals['Offer_Total_Amount'] = pd.to_numeric(deals['Offer_Total_Amount'], errors='coerce')

# add mode to NaN if product is known

unique_products = deals['Product'].dropna().unique()
for product in unique_products:
    product_group = deals.loc[deals['Product'] == product, 'Offer_Total_Amount']
    if not product_group.mode().empty:
        mode_val = product_group.mode().iloc[0]
        deals.loc[deals['Product'] == product, 'Offer_Total_Amount'] = deals.loc[deals['Product'] == product, 'Offer_Total_Amount'].fillna(mode_val)

# for unknown products not enough info to refill but to support format will add flag as 1000000

deals['Offer_Total_Amount'] = deals['Offer_Total_Amount'].fillna(1000000)

# print(deals['Offer_Total_Amount'].unique())

In [48]:
# 2.17. Evaluate what a pass means & Make a decision:
# City

# print(deals['City'].unique())

deals['City'] = deals['City'].fillna('Unknown')

# cleaning later

In [49]:
# 2.18. Evaluate what a pass means & Make a decision:
# Level_Of_German

# print(deals['Level_Of_German'].unique())

deals['Level_Of_German'] = deals['Level_Of_German'].fillna('Unknown')

# cleaning later

In [50]:
# 3. Converting columns to correct types

# numbers
deals['Deals_ID'] = deals['Deals_ID'].astype(str)
deals['Contact_Name'] = deals['Contact_Name'].astype(str)
deals['Initial_Amount_Paid'] = deals['Initial_Amount_Paid'].astype(int)
deals['Offer_Total_Amount'] = deals['Offer_Total_Amount'].astype(int)
deals['Course_Duration'] = deals['Course_Duration'].astype(int)

# dates
deals['Closing_Date'] = pd.to_datetime(deals['Closing_Date'], errors='coerce', dayfirst=True)
deals['Created_Time'] = pd.to_datetime(deals['Created_Time'], errors='coerce', dayfirst=True)

# categories
deals_categorical_columns = ['Deals_Manager', 'Quality', 'Stage', 'Lost_Reason', 'Page', 'Source', 'Payment_Type', 'Product', 'Education_Type', 'City', 'Level_Of_German']
for col in deals_categorical_columns:
    deals[col] = deals[col].astype('category')

In [51]:
# 4.1. Check for incorrect or illogical values
# deleting spaces in categories

for col in deals_categorical_columns:
    if deals[col].dtype == 'category':
        deals[col] = deals[col].str.strip()

In [52]:
# 4.2. Check for incorrect or illogical values
# check for negative values in course duration

negative_course_duration = deals[deals['Course_Duration'] < 0]
# if not negative_course_duration.empty:
#     print(f'Found {len(negative_duration)} cases with negative course duration')
# else:
#     print('Cases with negative course duration not found') # Cases with negative course duration not found

In [53]:
# 4.3. Check for incorrect or illogical values
# check for dates

deals_invalid_dates = deals[deals['Created_Time'] < deals['Closing_Date']]
if not invalid_dates.empty:
    print(f'Found {len(deals_invalid_dates)} invalid dates')
else:
    print('All dates are valid') # All dates are valid

All dates are valid


In [54]:
# 4.3. Check for incorrect or illogical values
# check for payments

deals_invalid_amounts = deals[deals['Offer_Total_Amount'] < deals['Initial_Amount_Paid']]
if not deals_invalid_amounts.empty:
    print(f'Found {len(deals_invalid_amounts)} invalid amounts')
    deals.loc[deals['Initial_Amount_Paid'] > deals['Offer_Total_Amount'], 'Initial_Amount_Paid'] = deals['Offer_Total_Amount']
else:
    print('All amounts are valid')

Found 15788 invalid amounts


In [55]:
# 5. Authentication of unique id

total_deals_rows = len(deals)
unique_deals_ids = deals['Deals_ID'].nunique()

if total_deals_rows == unique_deals_ids:
    print('All ID deals are unique')
else:
    print(f'Found duplicates in ID deals. Number of duplicates: {total_deals_rows - unique_deals_ids}') # Found duplicates in ID deals. Number of duplicates: 11480

total_deals_rows = len(deals)
unique_deals_contacts = deals['Contact_Name'].nunique()

if total_deals_rows == unique_deals_contacts:
    print('All contact names are unique')
else:
    print(f'Found duplicates in contact names. Number of duplicates: {total_deals_rows - unique_deals_contacts}') # Found duplicates in contact names. Number of duplicates: 11939

# contact could buy several courses, fill in applications from different sources
# duplicate deals may occur due to recording each time in the data system when changing, for example, stage

Found duplicates in ID deals. Number of duplicates: 11480
Found duplicates in contact names. Number of duplicates: 11939


In [56]:
# 6.1. Categorical feature processing
# Reducing number of categories
# Lost_Reason 

# print(deals['Lost_Reason'].unique())

category_mapping = {'Expensive': 'Financial Reasons', 'The contract did not fit': 'Financial Reasons', 'Gutstein refusal': 'Financial Reasons', 'Thought for free': 'Financial Reasons',
                    'Inadequate': 'Skill Issues', 'Does not know how to use a computer': 'Skill Issues', 'Does not speak English': 'Skill Issues',
                    "Doesn't Answer": 'No Response', 'Stopped Answering': 'No Response',
                    'needs time to think': 'Personal Issues', 'Not for myself': 'Personal Issues', 'Conditions are not suitable': 'Personal Issues',
                    'Considering a different direction in IT': 'Refusal', 'Changed Decision': 'Refusal', "Didn't leave an application": 'Refusal', 'Went to Rivals': 'Refusal',
                    'Invalid number': 'Technical Issues', 
                    'Non target': 'Other', 'Refugee': 'Other', 'Next stream': 'Other', 'Unknown': 'Other'}

deals['Lost_Reason_Combined'] = deals['Lost_Reason'].map(category_mapping)

# print(deals['Lost_Reason_combined'].unique())

In [57]:
# 6.2. Categorical feature processing
# Reducing number of categories
# Stage 

# print(deals['Stage'].unique())

category_stage_mapping = {'New Lead': 'New Lead', 
                          'Need a consultation': 'In Progress', 'Need to Call': 'In Progress', 'Need to Call - Sales': 'In Progress', 
                          'Call Delayed': 'In Progress', 'Qualificated': 'In Progress', 'Registered on Webinar': 'In Progress',
                          'Test Sent': 'In Progress', 'Registered on Offline Day': 'In Progress', 'Free Education': 'In Progress',
                          'Lost': 'Lost',
                          'Waiting For Payment': 'Waiting For Payment',
                          'Payment Done': 'Payment Done'}

deals['Stage_Combined'] = deals['Stage'].map(category_stage_mapping)

# print(deals['Stage_Сombined'].unique())

In [58]:
# 6.3. Categorical feature processing
# City cleaning

# print(deals['City'].unique())

deals['City'] = deals['City'].replace('-', 'Unknown')
deals.loc[deals['City'].str.contains(r'\d', na=False), 'City'] = 'Unknown' #remove any complicated data with numbers
deals.loc[deals['City'].str.contains(',', na=False), 'City'] = 'Unknown' #remove any complicated data with unnecessary info
deals['City'] = deals['City'].str.replace(r'\(.*?\)', '', regex=True) #remove any complicated data with unnecessary info

# with increasing amount of the data at least with each next stream, role of mistypes will increasse too. 
# there is universal solution is needed such as special library
# long (~20-30 mins) process but is worth it

from opencage.geocoder import OpenCageGeocode

geokey_API = '5bfa60fd7b464d17b3031543f79ba9aa'
geocoder = OpenCageGeocode(geokey_API) # free source

city_cache = {} #create dict to save time&memory

def normalize_city(city_name):
    if city_name in city_cache:
        return city_cache[city_name]
    location = geocoder.geocode(city_name, language='en')
    normilized_city_name = 'Unknown'
    if location and len(location) > 0:
        normilized_city_name = location[0]['components'].get('city', 'Unknown')
        if normilized_city_name == 'Unknown':
            normilized_city_name = location[0]['components'].get('town', 'Unknown')
                
    city_cache[city_name] = normilized_city_name
    return normilized_city_name

deals['City'] = deals['City'].apply(normalize_city)

# due to different languagues, there is many non standard letters that are not avaliable for everyone
# now may be kept
# better use universal solution as library to avoid problems with special symbols

import unidecode

deals['City'] = deals['City'].str.lower()

deals['City'] = deals['City'].apply(unidecode.unidecode) #special library that changed non standard to the most suitable standard symbol
deals['City'] = deals['City'].str.replace(r'[^a-zа-я\s]', ' ', regex=True).str.strip() #remove any other symbols

deals['City'] = deals['City'].str.capitalize()

# print(deals['City'].unique())

In [59]:
# 6.4. Categorical feature processing
# Level_Of_German cleaning

# print(deals['Level_Of_German'].unique())

# sometimes there is info together about german&english. decided to keep as a true with %%probability of mistake.
# especially because people sometimes evaluate their level independently also with error

deals['Level_Of_German'] = np.where(deals['Level_Of_German'].str.contains(r'[12]', na=False), deals['Level_Of_German'],'Unknown')

deals.loc[deals['Level_Of_German'] != 'Unknown', 'Level_Of_German'] = deals.loc[deals['Level_Of_German'] != 'Unknown', 'Level_Of_German'].str.lower()
deals.loc[deals['Level_Of_German'] != 'Unknown', 'Level_Of_German'] = deals.loc[deals['Level_Of_German'] != 'Unknown', 'Level_Of_German'].str.replace('а', 'a').str.replace('f', 'a').str.replace('в', 'b').str.replace('б', 'b').str.replace('с', 'c')
deals.loc[deals['Level_Of_German'] != 'Unknown', 'Level_Of_German'] = deals.loc[deals['Level_Of_German'] != 'Unknown', 'Level_Of_German'].str.extract(r'([abc][12](?:\-[abc][12])?)', expand=False)
deals['Level_Of_German'] = deals['Level_Of_German'].fillna('Unknown')

# print(deals['Level_Of_German'].unique()) # b2-c2 level found

deals.loc[deals['Level_Of_German'] == 'b2-c2', 'Level_Of_German'] = 'Unknown'

In [60]:
# 6.5. Categorical feature processing

deals_categorical_columns = ['Deals_Manager', 'Deals_Campaign', 'Quality', 'Stage', 'Stage_Combined', 'Lost_Reason_Combined', 'Lost_Reason', 'Page', 'Source', 'Payment_Type', 'Product', 'Education_Type', 'City', 'Level_Of_German']
for col in deals_categorical_columns:
    deals[col] = deals[col].astype('category')

In [61]:
# 8. Creating flags 

deals['Is_Deal_Lost'] = (deals['Stage'] == 'Lost')

deals['Is_Payment_Done'] = (deals['Stage'] == 'Payment Done')

deals['Is_Initial_Payment_Missig'] = (deals['Stage'] == 'Payment Done')&(deals['Initial_Amount_Paid'].isnull())

In [62]:
# check info about dataset Deals & structure after cleaning & prep

deals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19773 entries, 0 to 21592
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype          
---  ------                     --------------  -----          
 0   Deals_ID                   19773 non-null  object         
 1   Deals_Manager              19773 non-null  category       
 2   Closing_Date               13105 non-null  datetime64[ns] 
 3   Quality                    19773 non-null  category       
 4   Stage                      19773 non-null  category       
 5   Lost_Reason                14368 non-null  category       
 6   Page                       19773 non-null  category       
 7   Deals_Campaign             19773 non-null  category       
 8   SLA                        19773 non-null  timedelta64[ns]
 9   Content                    19773 non-null  object         
 10  Term                       19773 non-null  object         
 11  Source                     19773 non-null  category       


## Cleanig of dataset Spend

In [63]:
# 1. info about dataset Spend & checking structure

spend.info()
print(spend.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20779 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         20779 non-null  datetime64[ns]
 1   Source       20779 non-null  object        
 2   Campaign     14785 non-null  object        
 3   Impressions  20779 non-null  int64         
 4   Spend        20779 non-null  float64       
 5   Clicks       20779 non-null  int64         
 6   AdGroup      13951 non-null  object        
 7   Ad           13951 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 1.3+ MB
        Date        Source               Campaign  Impressions  Spend  Clicks  \
0 2023-07-03    Google Ads         gen_analyst_DE            6   0.00       0   
1 2023-07-03    Google Ads  performancemax_eng_DE            4   0.01       1   
2 2023-07-03  Facebook Ads                    NaN            0   0.00       0 

In [64]:
# Renaming

spend.rename(columns={'Source' : 'Spend_Source', 'Campaign' : 'Spend_Campaign', 'AdGroup' : 'Ad_Group'}, inplace=True)

In [65]:
# Check for duplicates

# spend.duplicated().sum()  #917 duplicates
spend = spend.drop_duplicates()

In [66]:
# 2. Check for NaN

spend.isnull().sum()

Date                 0
Spend_Source         0
Spend_Campaign    5077
Impressions          0
Spend                0
Clicks               0
Ad_Group          5911
Ad                5911
dtype: int64

In [67]:
# 2.1. Evaluate what a pass means & Make a decision:
# Spend_Campaign

# print(spend['Spend_Campaign'].unique())

# not enough info to add "valuable values"

spend['Spend_Campaign'] = spend['Spend_Campaign'].fillna('Unknown')

In [68]:
# 2.2. Evaluate what a pass means & Make a decision:
# Ad_Group

# print(spend['Ad_Group'].unique())

# not enough info to add "valuable values"

spend['Ad_Group'] = spend['Ad_Group'].fillna('Unknown')

In [69]:
# 2.3. Evaluate what a pass means & Make a decision:
# Ad

print(spend['Ad'].unique())

# not enough info to add "valuable values"

spend['Ad'] = spend['Ad'].fillna('Unknown')

[nan 'b3' 'b1' 'b4' 'b2' 'v2' 'v1' 'b4com' 'b3com' 'b2com' 'b1com' 'v6com'
 'v5' 'v4com' 'v3com' 'v5com' 'ad4' 'ad1' 'ad2' 'ad3' 'v8com' 'v7com'
 'ad6' 'ad5' 'bloggersvideo1com' 'v9com' 'ad9' 'ad8' 'ad_blogger_1'
 'ad_blogger_2' 'ad7' 'web_b3' 'web_b5' 'web_b1' 'web_b4' 'web_b2'
 'ad_blogger_3' 'v10com' 'bloggersvideo2com' 'b5' 'b6' 'b8' 'b7' 'v3'
 'v10' 'v12' 'v11com' 'v11' 'ad_gov_1' 'ad_da_1' 'b3comwebdev'
 'bloggersvideo2comwebdev' 'v11comwebdev' 'b1comwebdev' 'b2comwebdev'
 'bloggersvideo4com' 'bloggersvideo3com' 'bloggersvideo5' 'promo2'
 'promo1' 'ad_blogger_4' 'bloggersvideo4' 'b10' 'b11' 'b12' 'ad_blogger_6'
 'promo3' 'b15blackfriday' 'b14blackfriday' 'b13blackfriday' 'b7webinar'
 'b6webinar' 'b4webinar' 'b5webinar' 'bloggersvideo6blackfriday'
 'bloggersvideo6webinar' 'bloggersvideo7blackfriday'
 'bloggersvideo7webinar' 'bloggersvideo8webinar' 'v7blackfriday'
 'b9offlinewebinar' 'b10offlinewebinar' 'b8offlinewebinar'
 'bloggersvideo9com' 'bloggersvideo5com' 'bloggersvideo8com'

In [70]:
# 3. Converting columns to correct types

categorical_columns_spend = ['Spend_Source', 'Spend_Campaign']
for col in categorical_columns_spend:
    spend[col] = spend[col].astype('category')

In [71]:
# 4.1. Check for incorrect or illogical values
# checking mistypes in categories

for col in categorical_columns_spend:
    print(spend[col].value_counts()) # just for info to check mistypes. for thousands of unique values other methods are needed, at least creating a dict and checking with it

Spend_Source
Facebook Ads      9569
Tiktok Ads        2985
Youtube Ads       1784
Google Ads        1266
Telegram posts     836
Webinar            766
Bloggers           632
SMM                571
Organic            514
CRM                355
Test               262
Partnership        234
Offline             61
Radio               27
Name: count, dtype: int64
Spend_Campaign
Unknown                        5077
12.07.2023wide_DE              2073
02.07.23wide_DE                1685
04.07.23recentlymoved_DE       1398
youtube_shorts_DE              1223
07.07.23LAL_DE                 1181
03.07.23women                  1171
12.09.23interests_Uxui_DE      1143
15.07.23b_DE                    529
24.09.23retargeting_DE          504
performancemax_eng_DE           355
20.03.2024wide_PL               240
30.11.23wide_DE                 233
05.07.23interests_DE            214
17.03.24wide_AT                 198
07.12.23test_DE                 176
20.03.24interests_WebDev_PL     169
discovery_DE

In [72]:
# 4.2. Check for incorrect or illogical values
# check for negative values in spend

negative_spend = spend[spend['Spend'] < 0]

# if not negative_spend.empty:
#     print(f'Found {len(negative_spend)} cases with negative spends')
# else:
#     print('Negative spends not found') # Negative spends not found

In [73]:
# 4.3. Check for incorrect or illogical values
# check for negative values in clicks

negative_clicks = spend[spend['Clicks'] < 0]

# if not negative_clicks.empty:
#     print(f'Found {len(negative_clicks)} cases with negative clicks')
# else:
#     print('Negative clicks not found') # Negative clicks not found

In [74]:
# 4.4. Check for incorrect or illogical values
# check for negative values in impressions

negative_impressions = spend[spend['Impressions'] < 0]

# if not negative_impressions.empty:
#     print(f'Found {len(negative_impressions)} cases with negative impressions')
# else:
#     print('Negative impressions not found') # Negative impressions not found

In [75]:
# 8. Creating flags 

spend['Is_Campaign_Unknown'] = (spend['Spend_Campaign'] == 'Unknown')
        
spend['Is_Spend_For_No_Clicks'] = (spend['Clicks'] == 0) & (spend['Spend'] > 0)

In [76]:
# check info about dataset Spend & structure after cleaning & prep

spend.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19862 entries, 0 to 20778
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    19862 non-null  datetime64[ns]
 1   Spend_Source            19862 non-null  category      
 2   Spend_Campaign          19862 non-null  category      
 3   Impressions             19862 non-null  int64         
 4   Spend                   19862 non-null  float64       
 5   Clicks                  19862 non-null  int64         
 6   Ad_Group                19862 non-null  object        
 7   Ad                      19862 non-null  object        
 8   Is_Campaign_Unknown     19862 non-null  bool          
 9   Is_Spend_For_No_Clicks  19862 non-null  bool          
dtypes: bool(2), category(2), datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.1+ MB


In [77]:
#export with keeping all set formats

cleaned_project_data = [calls, contacts, deals, spend]

with open('cleaned_project_data.pickle', 'wb') as f:
    pickle.dump(cleaned_project_data, f)

# with open("data.pickle", "rb") as f:
#     calls, contacts, deals, spend = pickle.load(f)